# Model Inputs

### Packages

In [1]:
# General
import pandas as pd
import numpy as np
import pandas as pd
import os
DATA_DIR = "~/Desktop/code/data/"

# Pipeline
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

from sklearn.decomposition import PCA 
import umap
import hdbscan

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Visualization
import plotly.express as px
import plotly.graph_objects as go

### Dataset

In [2]:
# Loading output dataframe from data.ipynb
df = pd.read_csv(os.path.join(DATA_DIR, "ml_input.csv"), low_memory=False)

# Filter non-specific ec numbers (e.g., 1.1.1,-)
filter = df["ec"].str.contains("-")
df = df[~filter]

# Format for machine learning
df = df[["taxon_id", "media_id", "ec"]].value_counts().reset_index()
df = df.pivot(index=["taxon_id", "media_id"], columns="ec", values="count")
df = df.fillna(0.0).reset_index()

# Filtering into training and test sets (database set vs. MAGs)
bins = df["media_id"].str.contains("unknown")
df_train = df[~bins]
df_test = df[bins]

# Model

### Training and test sets

In [3]:
# Split the datasets into training and test sets
TARGET = "media_id" # target label
RDM = 787 # seed for random_state

# Test case with MAGs
X_train = df_train.drop(["taxon_id", "media_id"], axis=1)
y_train = df_train[TARGET]
X_test = df_test.drop(["taxon_id", "media_id"], axis=1)
y_test = df_test[TARGET]

### Pipeline

In [4]:
# sklearn pipeline helps prevent data leakage; incorporate individual steps here
pipeline = Pipeline([
    ('scaler', StandardScaler()),    # Scale data (optional)
    ('pca', PCA(n_components=120)),   # Reduce dimensionality with PCA
    ('umap', umap.UMAP(              # Further reduce with UMAP
        metric="euclidean",
        n_components=2, #20 for clustering, 2 for visualization
        n_epochs=200, #200 recommended for large datasets, higher => stricter clustering
        random_state=RDM,
        n_jobs=1,
        n_neighbors=50,
        min_dist=0.2
        )) 
])

### Fit and transform

In [5]:
# Fit and transform the pipeline on the training and test data
X_train_transformed = pipeline.fit_transform(X_train) # fit pipeline and transform training data
X_test_transformed = pipeline.transform(X_test) # transform test data

### Optimizing n_clusters

In [6]:
# Silhouette coefficient method
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Test a range of clusters for their silhouette coefficients
clusters = []
for n_cluster in range(20, 100):
    kmeans = KMeans(n_clusters=n_cluster).fit(X_train_transformed)
    label = kmeans.labels_
    sil_coeff = silhouette_score(X_train_transformed, label, metric='euclidean')
    clusters.append({'n_clusters': n_cluster, 'coefficient': sil_coeff})

# Select the maximum coefficient
clusters = pd.DataFrame(clusters)
c = clusters.iloc[clusters["coefficient"].argmax()]["n_clusters"]
s = clusters["coefficient"].max()
n = int(c)

print("{} clusters returns a maximum Silhouette Coefficient of {}".format(n-1, s))

95 clusters returns a maximum Silhouette Coefficient of 0.6989219188690186


In [7]:
# Cluster
clusterer = KMeans(n_clusters=n, random_state=RDM)
cluster_labels = clusterer.fit_predict(X_train_transformed)
test_clusters = clusterer.predict(X_test_transformed)

### Metrics

In [8]:
# Initial dimensionality reduction performance (PCA)
pca = pipeline.named_steps['pca']
explained_variance = pca.explained_variance_ratio_
print(f"Total PCA explained variance: {explained_variance.sum()}")

# Further dimensionality reduction and clustering performance
silhouette_avg = silhouette_score(X_train_transformed, cluster_labels)
print(clusterer.labels_.max(), "KMeans training clusters")
print(f"Average Silhouette Score on KMeans clusters: {silhouette_avg}")
    # Silhouette score <=0.25 implies poor clustering, 0.25<x<=0.50 is fair, <50 is good

Total PCA explained variance: 0.8272532690570777
95 KMeans training clusters
Average Silhouette Score on KMeans clusters: 0.6939911246299744


### Visualization

In [9]:
# Training data
train_df = pd.DataFrame(X_train_transformed, columns=['Component 1', 'Component 2'])
train_df['Set'] = 'Training'
train_df['Target'] = y_train
train_df['Cluster'] = cluster_labels # only showing training clusters for now, mapping test over these
train_df['taxon_id'] = df_train["taxon_id"]

# Media cluster labels
media_clusters = pd.read_csv(os.path.join(DATA_DIR, "comp-model.csv"))
media_clusters = media_clusters[["media_id", "Cluster"]]
media_clusters = media_clusters.rename(columns={"media_id": "Target", "Cluster": "mm_cluster"})

# Map labels
target_label_mapping = dict(zip(media_clusters['Target'], media_clusters['mm_cluster']))
train_df['mm_cluster'] = train_df['Target'].map(target_label_mapping)

train_df.head()

,Component 1,Component 2,Set,Target,Cluster,taxon_id,mm_cluster
0,-1.841079,27.584614,Training,J22,10,1002526,4.0
1,-10.670705,14.913761,Training,1a,0,1004166,11.0
2,20.357288,22.117355,Training,J181,36,1004261,4.0
3,20.372313,22.102917,Training,J455,36,1004261,2.0
4,-1.820855,27.620457,Training,J118,10,1005925,10.0


In [10]:
# Test data
test_df = pd.DataFrame(X_test_transformed, columns=['Component 1', 'Component 2'])
test_df['Set'] = 'Test'
test_df['Target'] = list(df_test['media_id'])
test_df['Cluster'] = test_clusters
test_df['taxon_id'] = list(df_test['taxon_id'])
test_df.head()

,Component 1,Component 2,Set,Target,Cluster,taxon_id
0,8.983127,6.885402,Test,unknown,82,KBase_derived_Bin.001.fastaBA_F_extracted_bins...
1,8.389124,8.266546,Test,unknown,65,KBase_derived_Bin.002.fastaBA_F_extracted_bins...
2,9.772591,6.769083,Test,unknown,82,KBase_derived_Bin.004.fastaBA_F_extracted_bins...
3,9.297498,6.872422,Test,unknown,82,KBase_derived_Bin.005.fastaBA_F_extracted_bins...
4,8.747047,7.127034,Test,unknown,82,KBase_derived_Bin.006.fastaBA_F_extracted_bins...


In [11]:
# Plot the training points
fig = px.scatter(
    data_frame = train_df,
    x = "Component 1",
    y = "Component 2",
    color = "Cluster",
    hover_data = ["taxon_id", "Target", "mm_cluster"],
    opacity = 0.3
) 

# Add the test points as a scatter trace
fig.add_trace(
    go.Scatter(
        x=test_df["Component 1"],  # x coordinates
        y=test_df["Component 2"],  # y coordinates
        mode='markers',
        marker=dict(
            color='black',  
            size=5,  
            opacity=1.0 
        ),
        text=test_df["Target"]  # marker hover text
    )
)

# Opacity slider
steps = []
for i in range(11):  # 0 to 1 in steps of 0.1
    step = dict(
        method="restyle",
        args=[{"marker.opacity": [i/20]}, [0]],  # Adjust opacity of the first trace only
        label=str(i/20)
    )
    steps.append(step)
sliders = [dict(
    active=3,  # initial value corresponding to opacity=0.3
    currentvalue={"prefix": "Opacity: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title="KMeans",
    template="plotly_white",
)

fig.show()

# Classifier

### Nearest Neighbors

In [12]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_transformed, cluster_labels)

print(neigh.predict(X_test_transformed))
distances, indices = neigh.kneighbors(X_test_transformed)

[82 65 82 82 82 65 82 82 65 82 82 65 81 82 85 65 85 65 44 82 82 82 65]


In [13]:
# Add nearest neighbor indices to test dataframe
output_df = test_df.copy()
nn_df = pd.DataFrame(indices)
output_df['indices'] = nn_df[nn_df.columns[0:]].apply(lambda x: ';'.join(x.dropna().astype(str)), axis=1)
#output_df["KNN_label"] = neigh.predict(X_test_transformed)

# Explode the indices
output_df["indices"] = output_df["indices"].str.split(';')
output_df = output_df.explode("indices")

# Add indices as column for the training dataframe
train_index = train_df.copy()
train_index["indices"] = train_index.index
train_index["indices"] = train_index["indices"].astype(str)

# Merge our dataframes to provide labels
left = output_df.copy()
right = train_index[["indices", "Target", "Cluster", "taxon_id"]]
right = right.rename(columns={"Target": "neighbor_media_id", "taxon_id": "neighbor_taxon_id", "Cluster": "neighbor_cluster"})
merged = pd.merge(left, right, on="indices", how="left")



#merged.to_csv(os.path.join(DATA_DIR, "ml-model-output.csv"), index=False)
merged.head()

,Component 1,Component 2,Set,Target,Cluster,taxon_id,indices,KNN_label,neighbor_media_id,neighbor_cluster,neighbor_taxon_id
0,8.983127,6.885402,Test,unknown,82,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,350,82,J112,82,119072
1,8.983127,6.885402,Test,unknown,82,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,2353,82,J58,82,1951
2,8.983127,6.885402,Test,unknown,82,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,6192,82,J112,82,867682
3,8.983127,6.885402,Test,unknown,82,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,4516,82,J597,82,440
4,8.983127,6.885402,Test,unknown,82,KBase_derived_Bin.001.fastaBA_F_extracted_bins...,7439,82,J118,82,"[{'NCBI tax id': 228, 'Matching level': 'speci..."


### Random Forest

In [14]:
y_train = cluster_labels
y_test = test_clusters

In [15]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=RDM
)

model.fit(X_train_transformed, y_train)

RandomForestClassifier(n_jobs=-1, random_state=787)

In [16]:
report = classification_report(
    y_true=y_test,
    y_pred=model.predict(X_test_transformed),
    zero_division="warn"
)

print(report)

              precision    recall  f1-score   support

          44       1.00      1.00      1.00         1
          65       1.00      1.00      1.00         7
          81       1.00      1.00      1.00         1
          82       1.00      1.00      1.00        12
          85       1.00      1.00      1.00         2

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



In [17]:
from sklearn import metrics

roc_score = metrics.roc_auc_score(
    y_true=y_test,
    y_score=model.predict_proba(X_test_transformed),
    labels=np.unique(y_train),
    average="weighted",
    multi_class="ovo"
)

print("ROC AUC score:", roc_score)

ROC AUC score: 1.0
